In [1]:
import pandas as pd

df_points = pd.read_csv('../data/final_df_points_18_21_class.csv')


In [2]:
df_points['idle_time'].max()

1439

In [3]:
df_points = df_points[(df_points['idle_time'] > 5)]
df_points

,bike_id,lat,lng,dt_start,dt_end,hex_id,hex_enc,in_zone,zone_name,zone_name_enc,...,wind_speed,humidity,year,month,day,start_min,over_night,idle_time_next_day,idle_time,idle_time_class
0,3694,51.323619,12.335887,1514764029,1514764799,881f1a8d91fffff,324,1,Pinke Zone Leipzig West 1€,4,...,7.20,76.0,2017,12,6,1427,1,821,12,1
1,2508,51.323619,12.335887,1514764035,1514764799,881f1a8d91fffff,324,1,Pinke Zone Leipzig West 1€,4,...,7.20,76.0,2017,12,6,1427,1,1439,12,1
3,3694,51.323619,12.335887,1514764800,1514814103,881f1a8d91fffff,324,1,Pinke Zone Leipzig West 1€,4,...,9.30,76.0,2018,1,0,0,0,0,821,4
4,2508,51.323619,12.335887,1514764800,1514851199,881f1a8d91fffff,324,1,Pinke Zone Leipzig West 1€,4,...,9.30,76.0,2018,1,0,0,1,878,1439,4
5,41851,51.332415,12.395158,1514764800,1514851199,881f1a1659fffff,137,1,Pinke Zone Leipzig Ost 1€,3,...,9.30,76.0,2018,1,0,0,1,82,1439,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543825,72058,51.317569,12.382730,1640897593,1640904368,881f1a1609fffff,102,1,Blaue Zone Leipzig Ost 0€,1,...,8.94,83.0,2021,12,3,1253,0,0,112,2
2543826,71949,51.334711,12.356763,1640897835,1640899343,881f1a1649fffff,130,1,Pinke Zone Leipzig Ost 1€,3,...,8.94,83.0,2021,12,3,1257,0,0,25,1
2543827,97785,51.367276,12.369792,1640898086,1640900985,881f1a8c8bfffff,265,1,Blaue Zone Leipzig Ost 0€,1,...,8.05,84.0,2021,12,3,1261,0,0,48,1
2543828,41738,51.366942,12.370322,1640900918,1640903516,881f1a8c8bfffff,265,1,Pinke Zone Leipzig Ost 1€,3,...,8.05,84.0,2021,12,3,1309,0,0,43,1


In [4]:
df_points['over_night'].value_counts()

0    1819614
1     641827
Name: over_night, dtype: int64

In [9]:
641827 / 1819614

0.352727006936636

def f(row):
    if row['idle_time'] < 1439:
        val = '1day'
    else:
        val = 'longer'
    return val
#
df_points['idle_time_class'] = df_points.apply(f, axis=1)

from matplotlib import pyplot as plt

fig = plt.plot(df_points.value_counts(['idle_time_class']).values)
df_points.value_counts(['idle_time_class']).values

In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import wandb

#idle_time_data = pd.read_csv('../data/final_nosplit_df_points_18_21_class.csv')

idle_time_data = df_points

TargetVariable = ['over_night']
Predictors = ['bike_id', 'lat', 'lng', 'temp', 'rain', 'snow', 'wind_speed', 'humidity', 'dt_start',
              'hex_enc', 'start_min', 'year', 'month', 'day', 'on_station', 'in_zone', 'zone_name_enc']

X = idle_time_data[Predictors].values
y = idle_time_data[TargetVariable].values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, shuffle=False)

In [6]:
from sklearn.metrics import f1_score

sweep_configuration = {
    "project": "Over-Night-RF-Class",
    "name": "my-awesome-sweep-recall-max",
    "metric": {"name": "macro_recall", "goal": "maximize"},
    "method": "random",
    "parameters": {
        "n_estimators": {
            "values": [8, 14, 19, 24, 32, 41, 54, 65, 78,86,98, 107, 114, 128]
        },
        "criterion": {
            "values": ['entropy', 'gini']
        },
        "max_depth": {
            "values": [2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 140, 160, 180, 200, None]
        },
        "bootstrap": {
            "values": [True, False]
        },
        "max_features": {
            "values": ['auto', 'sqrt', 'log2']
        },
        "min_samples_leaf": {
            "values": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60]
        },
        "min_samples_split": {
            "values": [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 18, 20, 25, 30, 40, 50, 60]
        }
    }
}

def eval_classification(y_test,y_pred,labels):
    # Metrics
    acc = accuracy_score(y_test, y_pred.ravel())
    macro_precision = precision_score(y_test.ravel(), y_pred.ravel(), average='macro', labels=labels)
    macro_recall = recall_score(y_test.ravel(), y_pred.ravel(), average='macro', labels=labels)

    macro_f1 = f1_score(y_test.ravel(), y_pred.ravel(), average='macro', labels=labels)

    print(acc)
    print(macro_precision)
    print(macro_recall)
    print(macro_f1)

    return acc, macro_precision, macro_recall, macro_f1

def my_train_func():
    wandb.init()

    _n_estimators = wandb.config.n_estimators
    _criterion = wandb.config.criterion
    _max_depth = wandb.config.max_depth
    _bootstrap = wandb.config.bootstrap
    _max_features = wandb.config.max_features
    _min_samples_leaf = wandb.config.min_samples_leaf
    _min_samples_split = wandb.config.min_samples_split

    model = RandomForestClassifier(n_estimators=_n_estimators,
                                   criterion=_criterion,
                                   max_depth=_max_depth,
                                   bootstrap=_bootstrap,
                                   max_features=_max_features,
                                   min_samples_leaf=_min_samples_leaf,
                                   min_samples_split=_min_samples_split,
                                   n_jobs=-1)

    model.fit(X_train, y_train.ravel())
    y_pred = model.predict(X_test)

    acc, macro_precision, macro_recall, macro_f1 = eval_classification(y_test,y_pred,[0,1])

    wandb.log({"accuracy": acc})
    wandb.log({"conf_matrix": wandb.plot.confusion_matrix(y_true=y_test.ravel(), preds=y_pred.ravel())})
    wandb.log({"macro_precision": macro_precision})
    wandb.log({"macro_recall": macro_recall})
    wandb.log({"macro_f1": macro_f1})

In [7]:
import os
os.environ["WANDB_SILENT"] = "true"

# INIT SWEEP
sweep_id_rfc = wandb.sweep(sweep_configuration, project="Over-Night-RF-Class")
# RUN SWEEP
wandb.agent(sweep_id_rfc, function=my_train_func)

Create sweep with ID: jqeo4owp
Sweep URL: https://wandb.ai/jonathanweske/Over-Night-RF-Class/sweeps/jqeo4owp
0.8193381949663816
0.7831929597497166
0.65631508720126
0.6830674488216569

0.8190294338702797
0.7763081799906296
0.6615375066091416
0.6879369927956008

0.8015194296045014
0.7955695200583721
0.5934638148384651
0.6036161921052612

0.8182372179000182
0.772782321238447
0.6621480817145933
0.6881736521152091

0.8200004062646001
0.782780284063931
0.6590315548343167
0.6859606990140243

0.8182778443600317
0.7754604902741928
0.6595263246331506
0.6857353824006437

0.8111275873976721
0.7868729728305055
0.6262480663448884
0.6481016801846868

0.8196672692924902
0.7851334492082715
0.6558570063605254
0.6827454171384448

0.8070811919803368
0.7712179249754996
0.6224095528029611
0.6427672326745681

0.8194438237624164
0.7700213244417228
0.6702675802080489
0.6960147979957878

0.8200125942026042
0.7823076123295252
0.6594940075063245
0.6864086986146514

0.8085681204168275
0.7619310178198626
0.63515387